In [94]:
np.set_printoptions(precision=6)

In [179]:
import fileinput
import re
import numpy as np

file_base = '/home/lindsayad/scale/C834MNYCP02/SCALE-6.2.1/lindsayad/io/arfc_reactor/calc_xsecs_provided_buckling_'

fuel_temps = [fuel_temp for fuel_temp in range(800, 1050 + 50, 50)]
mod_temps = [mod_temp for mod_temp in range(800, 1050 + 50, 50)]

# # Uncomment this if wanting to create a 2D group cross section dependence on both fuel and moderator temperatures
# # Would have to write more code to get this into a good 2D format
# all_files = [file_base + str(fuel_temp) + '_mod_' + str(mod_temp) + '.out' for mod_temp in mod_temps for fuel_temp in fuel_temps]

all_files = [file_base + str(mod_temp) + '_mod_' + str(mod_temp) + '.out' for mod_temp in mod_temps]

num_groups = 2
num_precursor_groups = 6

fuel_xsecs = []
mod_xsecs = []
fuel_scatter_xsecs = []
mod_scatter_xsecs = []
fuel_inv_vel = []
fuel_lambda = []
fuel_beta = []
fuel_chi = []
mod_inv_vel = []
mod_lambda = []
mod_beta = []
mod_chi = []

for file_name in all_files:

    file = open(file_name, 'r')

    # Collect a whole lot of fuel transport data
    for line in file:
        if re.search(r'Group.*Total.*Transport', line):
            break
    file.readline()
    fuel_current_temp_xsecs = []
    for i in range(num_groups):
        fuel_current_temp_xsecs.append([float(i) for i in file.readline().split()])
    fuel_xsecs.append(fuel_current_temp_xsecs)

    # Collect fuel recipvel
    for line in file:
        if re.search(r'Group.*Inv\. Velocity', line):
            break
    file.readline()
    file.readline()
    current_temp_fuel_inv_vel = []
    for i in range(num_groups):
        inv_velocity_line_string = file.readline().split()
        current_temp_fuel_inv_vel.append(inv_velocity_line_string[1])
    fuel_inv_vel.append(current_temp_fuel_inv_vel)

    # Collect fuel scattering data
    for line in file:
        if re.search(r'P0 Scattering Matrix', line):
            break
    file.readline()
    file.readline()
    current_temp_fuel_scatter_xsecs = []
    first_scatter_line = file.readline().split()
    current_temp_fuel_scatter_xsecs.append([float(string) for index, string in enumerate(first_scatter_line) \
                                       if index in range(2, len(first_scatter_line))])
    for i in range(1, num_groups):
        next_scatter_line = file.readline().split()
        current_temp_fuel_scatter_xsecs.append([float(string) for index, string in enumerate(next_scatter_line) \
                                           if index in range(1, len(next_scatter_line))])
    fuel_scatter_xsecs.append(current_temp_fuel_scatter_xsecs)

    # Collect fuel chi
    for line in file:
        if re.search(r'Prompt fission fraction', line):
            break
    file.readline()
    file.readline()
    current_temp_fuel_chi = []
    for i in range(num_groups):
        chi_line_string = file.readline().split()
        current_temp_fuel_chi.append(chi_line_string[1])
    fuel_chi.append(current_temp_fuel_chi)

    # Collect fuel lambda and beta
    for line in file:
        if re.search(r'Lambda', line):
            break
    file.readline()
    current_temp_fuel_lambda = []
    current_temp_fuel_beta = []
    for i in range(num_precursor_groups):
        decay_line_string = file.readline().split()
        current_temp_fuel_lambda.append(decay_line_string[1])
        current_temp_fuel_beta.append(decay_line_string[2])
    fuel_lambda.append(current_temp_fuel_lambda)
    fuel_beta.append(current_temp_fuel_beta)

    # Collect a whole lot of moderator transport data
    for line in file:
        if re.search(r'Group.*Total.*Transport', line):
            break
    file.readline()
    mod_current_temp_xsecs = []
    for i in range(num_groups):
        mod_current_temp_xsecs.append([float(i) for i in file.readline().split()])
    mod_xsecs.append(mod_current_temp_xsecs)

    # Collect mod recipvel
    for line in file:
        if re.search(r'Group.*Inv\. Velocity', line):
            break
    file.readline()
    file.readline()
    current_temp_mod_inv_vel = []
    for i in range(num_groups):
        inv_velocity_line_string = file.readline().split()
        current_temp_mod_inv_vel.append(inv_velocity_line_string[1])
    mod_inv_vel.append(current_temp_mod_inv_vel)

    # Collect moderator scattering data
    for line in file:
        if re.search(r'P0 Scattering Matrix', line):
            break
    file.readline()
    file.readline()
    current_temp_mod_scatter_xsecs = []
    first_scatter_line = file.readline().split()
    current_temp_mod_scatter_xsecs.append([float(string) for index, string in enumerate(first_scatter_line) \
                                       if index in range(2, len(first_scatter_line))])
    for i in range(1, num_groups):
        next_scatter_line = file.readline().split()
        current_temp_mod_scatter_xsecs.append([float(string) for index, string in enumerate(next_scatter_line) \
                                           if index in range(1, len(next_scatter_line))])
    mod_scatter_xsecs.append(current_temp_mod_scatter_xsecs)

    # Collect mod chi
    for line in file:
        if re.search(r'Prompt fission fraction', line):
            break
    file.readline()
    file.readline()
    current_temp_mod_chi = []
    for i in range(num_groups):
        chi_line_string = file.readline().split()
        current_temp_mod_chi.append(chi_line_string[1])
    mod_chi.append(current_temp_mod_chi)

    # Collect mod lambda and beta
    for line in file:
        if re.search(r'Lambda', line):
            break
    file.readline()
    current_temp_mod_lambda = []
    current_temp_mod_beta = []
    for i in range(num_precursor_groups):
        decay_line_string = file.readline().split()
        current_temp_mod_lambda.append(decay_line_string[1])
        current_temp_mod_beta.append(decay_line_string[2])
    mod_lambda.append(current_temp_mod_lambda)
    mod_beta.append(current_temp_mod_beta)
# end for file_name

fuel_array = np.array(fuel_xsecs)
mod_array = np.array(mod_xsecs)
fuel_scatter_array = np.array(fuel_scatter_xsecs)
mod_scatter_array = np.array(mod_scatter_xsecs)

fuel_sig_out_scatter = np.zeros((len(fuel_temps), num_groups))
mod_sig_out_scatter = np.zeros((len(fuel_temps), num_groups))
for i in range(fuel_scatter_array.shape[0]):
    for j in range(num_groups):
        # for k in num_groups:
        fuel_sig_out_scatter[i][j] = sum([to_k_from_j_scatter_xsec for k, to_k_from_j_scatter_xsec \
                                    in np.ndenumerate(fuel_scatter_array[i][j]) if k[0] != j])
        mod_sig_out_scatter[i][j] = sum([to_k_from_j_scatter_xsec for k, to_k_from_j_scatter_xsec \
                                    in np.ndenumerate(mod_scatter_array[i][j]) if k[0] != j])


fuel_nsf = fuel_array[:, :, 5]
fuel_sig_f = fuel_array[:, :, 7]
fuel_sig_a = fuel_array[:, :, 4]
fuel_sig_r = fuel_sig_a + fuel_sig_out_scatter
fuel_nubar = fuel_nsf / fuel_sig_f
fuel_fisse = fuel_array[:, :, 6] / fuel_sig_f / 1.6e-19 / 1e6
fuel_diff_coeff = 1. / (fuel_array[:, :, 2] * 3.)
fuel_flux = fuel_array[:, :, 8]
fuel_inv_vel = np.array(fuel_inv_vel)
fuel_lambda = np.array(fuel_lambda)
fuel_beta = np.array(fuel_beta)
fuel_chi = np.array(fuel_chi)

mod_nsf = mod_array[:, :, 5]
mod_sig_f = mod_array[:, :, 7]
mod_sig_a = mod_array[:, :, 4]
mod_sig_r = mod_sig_a + mod_sig_out_scatter
mod_nubar = mod_nsf * 0
mod_fisse = mod_array[:, :, 6] * 0
mod_diff_coeff = 1. / (mod_array[:, :, 2] * 3.)
mod_flux = mod_array[:, :, 8]
mod_inv_vel = np.array(mod_inv_vel)
mod_lambda = np.array(mod_lambda)
mod_beta = np.array(mod_beta)
mod_chi = np.array(mod_chi)


In [180]:
class XsecData():
    def __init__(self, temperature, data, xsec_name, mat_name, out_prefix="newt"):
        self.name = xsec_name.upper()
        self.temperature = np.array(temperature)
        self.data = data
        self.dim = len(data.shape) - 1
        self._error_check()
        self.out_name = out_prefix + "_" + mat_name + "_" + self.name + ".txt"
        
    def _error_check(self):
        if len(self.temperature.shape) > 1:
            raise RuntimeError("Temperature should only be a 1D array or list")
        if self.temperature.shape[0] != self.data.shape[0]:
            raise RuntimeError("Temperature and data are not the same length")
        if self.dim < 1 or self.dim > 2:
            raise RuntimeError("We expect 1D sets like fission cross section or 2D sets like "
                               "scattering matrices")
        valid_names = ["FLUX", "REMXS", "FISSXS", "NUBAR", "NSF", "FISSE", "DIFFCOEF", \
          "RECIPVEL", "CHI", "GTRANSFXS", "BETA_EFF", "DECAY_CONSTANT"]
        if self.name not in valid_names:
            raise RuntimeError("Specified xsec name does not match any expected by Moltres")
        
    def write_moltres_input_file(self):
        with open(self.out_name, 'w') as f:
            for i in range(self.temperature.shape[0]):
                if self.dim == 1:
                    data_string_list = [" " + str(j) for j in self.data[i]]
                if self.dim == 2:
                    data_string_list = []
                    for j in range(self.data[i].shape[0]):
                        for k in self.data[i][j]:
                            data_string_list.append(" " + str(k))
                write_string = str(self.temperature[i]) + "".join(data_string_list) + "\n"
                f.write(write_string)        

In [181]:
fuel_data_dict = {"FLUX" : fuel_flux, "REMXS" : fuel_sig_r, "NUBAR" : fuel_nubar, "NSF" : fuel_nsf, "FISSE" : fuel_fisse, "DIFFCOEF" : fuel_diff_coeff, "RECIPVEL" : fuel_inv_vel, "CHI" : fuel_chi, "GTRANSFXS" : fuel_scatter_array, "BETA_EFF" : fuel_beta, "DECAY_CONSTANT" : fuel_lambda}
mod_data_dict = {"FLUX" : mod_flux, "REMXS" : mod_sig_r, "NUBAR" : mod_nubar, "NSF" : mod_nsf, "FISSE" : mod_fisse, "DIFFCOEF" : mod_diff_coeff, "RECIPVEL" : mod_inv_vel, "CHI" : mod_chi, "GTRANSFXS" : mod_scatter_array, "BETA_EFF" : mod_beta, "DECAY_CONSTANT" : mod_lambda}

for name, data in fuel_data_dict.items():
    XsecData(mod_temps, data, name, "fuel").write_moltres_input_file()
for name, data in mod_data_dict.items():
    XsecData(mod_temps, data, name, "mod").write_moltres_input_file()

In [175]:
stupid_dictionary = {"john" : 1, "dan" : 2}

In [176]:
stupid_dictionary["john"]

1

In [170]:
data_string_list = []
for i in range(fuel_scatter_array.shape[0]):
    for j in range(fuel_scatter_array[i].shape[0]):
        for k in fuel_scatter_array[i][j]:
            data_string_list.append(" " + str(k))
write_string = "".join(data_string_list)
print(write_string)

 0.30693 0.00166688 0.000972376 0.288354 0.303973 0.00171639 0.00108954 0.285505 0.300905 0.0017751 0.0012223 0.282539 0.29791 0.00185298 0.00139188 0.279675 0.294802 0.00194277 0.00158033 0.276692 0.291761 0.00204546 0.00178887 0.27376


In [165]:
for i in range(fuel_scatter_array.shape[0]):
    for j in range(fuel_scatter_array[i].shape[0]):
        for k in range(fuel_scatter_array[i][j].shape[0]):
            print(k)

0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1


In [152]:
print(fuel_scatter_array)

[[[ 0.30693   0.001667]
  [ 0.000972  0.288354]]

 [[ 0.303973  0.001716]
  [ 0.00109   0.285505]]

 [[ 0.300905  0.001775]
  [ 0.001222  0.282539]]

 [[ 0.29791   0.001853]
  [ 0.001392  0.279675]]

 [[ 0.294802  0.001943]
  [ 0.00158   0.276692]]

 [[ 0.291761  0.002045]
  [ 0.001789  0.27376 ]]]


In [162]:
for j in range(fuel_scatter_array[0].shape[0]):
    print(j)

0
1


In [155]:
print(type(fuel_scatter_array[0]))

<class 'numpy.ndarray'>


In [173]:
scatter_object = XsecData(mod_temps, fuel_scatter_array, "GTRANSFXS", "fuel")

scatter_object.write_moltres_input_file()

In [174]:
fission_object = XsecData(mod_temps, fuel_sig_f, "FISSXS", "fuel")

fission_object.write_moltres_input_file()

In [127]:
fake_temp1 = [100, 200, 300]
fake_temp2 = [[100, 200], [300, 400]]

In [131]:
fake_f_object = XsecData(fake_temp2, fuel_sig_f, "")

RuntimeError: Temperature should only be a 1D array or list

In [118]:
print(fuel_chi)
print(mod_chi)

[['1.00000E+00' '0.00000E+00']
 ['1.00000E+00' '0.00000E+00']
 ['1.00000E+00' '0.00000E+00']
 ['1.00000E+00' '0.00000E+00']
 ['1.00000E+00' '0.00000E+00']
 ['1.00000E+00' '0.00000E+00']]
[['0.00000E+00' '0.00000E+00']
 ['0.00000E+00' '0.00000E+00']
 ['0.00000E+00' '0.00000E+00']
 ['0.00000E+00' '0.00000E+00']
 ['0.00000E+00' '0.00000E+00']
 ['0.00000E+00' '0.00000E+00']]


In [115]:
print(mod_fisse)

[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]


In [110]:
print(fuel_inv_vel)
print(fuel_beta)
print(fuel_lambda)
print(mod_inv_vel)
print(mod_beta)
print(mod_lambda)

[['8.82678E-08' '2.20878E-06']
 ['8.89665E-08' '2.15884E-06']
 ['8.97751E-08' '2.11260E-06']
 ['9.07549E-08' '2.07008E-06']
 ['9.18915E-08' '2.03057E-06']
 ['9.31834E-08' '1.99373E-06']]
[['2.71651E-04' '9.00498E-04' '7.85468E-04' '9.07969E-04' '1.82857E-04'
  '1.10602E-04']
 ['2.71667E-04' '9.00830E-04' '7.85588E-04' '9.07886E-04' '1.82754E-04'
  '1.10593E-04']
 ['2.71682E-04' '9.01172E-04' '7.85709E-04' '9.07788E-04' '1.82643E-04'
  '1.10582E-04']
 ['2.71697E-04' '9.01513E-04' '7.85829E-04' '9.07697E-04' '1.82535E-04'
  '1.10572E-04']
 ['2.71713E-04' '9.01867E-04' '7.85953E-04' '9.07593E-04' '1.82419E-04'
  '1.10561E-04']
 ['2.71728E-04' '9.02214E-04' '7.86075E-04' '9.07491E-04' '1.82305E-04'
  '1.10550E-04']]
[['1.25852E-02' '3.38917E-02' '1.38636E-01' '3.23935E-01' '1.15104E+00'
  '2.61020E+00']
 ['1.25853E-02' '3.38904E-02' '1.38638E-01' '3.23941E-01' '1.15093E+00'
  '2.60957E+00']
 ['1.25854E-02' '3.38891E-02' '1.38640E-01' '3.23948E-01' '1.15082E+00'
  '2.60891E+00']
 ['1.25855E

In [103]:
print(fuel_sig_out_scatter)
print(fuel_sig_a)
print(fuel_sig_r)
print(mod_sig_out_scatter)
print(mod_sig_a)
print(mod_sig_r)

[[ 0.001667  0.000972]
 [ 0.001716  0.00109 ]
 [ 0.001775  0.001222]
 [ 0.001853  0.001392]
 [ 0.001943  0.00158 ]
 [ 0.002045  0.001789]]
[[ 0.009272  0.041553]
 [ 0.009254  0.040238]
 [ 0.009231  0.038997]
 [ 0.009218  0.037851]
 [ 0.009201  0.036763]
 [ 0.009183  0.035748]]
[[ 0.010938  0.042526]
 [ 0.01097   0.041328]
 [ 0.011006  0.040219]
 [ 0.011071  0.039243]
 [ 0.011143  0.038344]
 [ 0.011228  0.037537]]
[[ 0.003972  0.001625]
 [ 0.004113  0.001867]
 [ 0.004278  0.002144]
 [ 0.004491  0.002505]
 [ 0.004736  0.002912]
 [ 0.005015  0.003368]]
[[  1.399310e-05   1.558530e-04]
 [  1.405210e-05   1.522490e-04]
 [  1.411390e-05   1.488600e-04]
 [  1.418810e-05   1.457430e-04]
 [  1.426690e-05   1.428580e-04]
 [  1.436200e-05   1.401750e-04]]
[[ 0.003986  0.001781]
 [ 0.004127  0.002019]
 [ 0.004292  0.002293]
 [ 0.004505  0.002651]
 [ 0.00475   0.003055]
 [ 0.005029  0.003508]]


Remaining cross section data to parse:

- recipvel
- chi
- betta_eff
- decay_constant

|0|1|2|3|4|5|6|7|8|
|-|-|-|-|-|-|-|-|-|
|group_number|sig_t|sig_tr|sig_a_weird|sig_a|nsf|kappa\*fission|sig_f|flux|


In [102]:
.001667 + .009272

0.010939

In [90]:
fuel_scatter_array

array([[[ 0.307,  0.002],
        [ 0.001,  0.288]],

       [[ 0.304,  0.002],
        [ 0.001,  0.286]],

       [[ 0.301,  0.002],
        [ 0.001,  0.283]],

       [[ 0.298,  0.002],
        [ 0.001,  0.28 ]],

       [[ 0.295,  0.002],
        [ 0.002,  0.277]],

       [[ 0.292,  0.002],
        [ 0.002,  0.274]]])

In [85]:
fuel_scatter_array[0][0]

array([ 0.307,  0.002])

In [82]:
print(fuel_nsf[0])

[ 0.008  0.062]


In [83]:
print(fuel_nsf[0].size)

2


In [81]:
print(mod_nsf.shape)

(6, 2)


|0|1|2|3|4|5|6|7|8|
|-|-|-|-|-|-|-|-|-|
|group_number|sig_t|sig_tr|sig_a_weird|sig_a|nsf|kappa\*fission|sig_f|flux|


In [79]:
print(fuel_array.shape)
print(fuel_array[:, 0, 1])
print(mod_array.shape)
print(mod_array[:, 0, 1])
print(fuel_scatter_array.shape)
print(mod_scatter_array.shape)

(6, 2, 9)
[ 0.318  0.315  0.312  0.309  0.306  0.303]
(6, 2, 9)
[ 0.395  0.395  0.395  0.395  0.395  0.395]
(6, 2, 2)
(6, 2, 2)


In [80]:
print(fuel_scatter_array[:, :, :])
print(mod_scatter_array[:, :, :])

[[[ 0.307  0.002]
  [ 0.001  0.288]]

 [[ 0.304  0.002]
  [ 0.001  0.286]]

 [[ 0.301  0.002]
  [ 0.001  0.283]]

 [[ 0.298  0.002]
  [ 0.001  0.28 ]]

 [[ 0.295  0.002]
  [ 0.002  0.277]]

 [[ 0.292  0.002]
  [ 0.002  0.274]]]
[[[ 0.391  0.004]
  [ 0.002  0.453]]

 [[ 0.391  0.004]
  [ 0.002  0.453]]

 [[ 0.391  0.004]
  [ 0.002  0.453]]

 [[ 0.391  0.004]
  [ 0.003  0.453]]

 [[ 0.391  0.005]
  [ 0.003  0.453]]

 [[ 0.39   0.005]
  [ 0.003  0.452]]]


In [64]:
print(fuel_array[:, :, 1])
print(mod_array[:, :, 1])

[[ 0.318  0.331]
 [ 0.315  0.327]
 [ 0.312  0.323]
 [ 0.309  0.319]
 [ 0.306  0.315]
 [ 0.303  0.311]]
[[ 0.395  0.455]
 [ 0.395  0.455]
 [ 0.395  0.455]
 [ 0.395  0.455]
 [ 0.395  0.456]
 [ 0.395  0.456]]


In [67]:
a = np.array([[1,2,3],
              [4,5,6]])
b = np.array([[3,2,4],
              [5,6,7]])
c = np.array([[7,8,9],
              [10,11,12]])
print(c.shape)
newarray = np.concatenate([a, b, c])
print(newarray.shape)

(2, 3)
(6, 3)


In [7]:
filename = '/home/lindsayad/scale/C834MNYCP02/SCALE-6.2.1/lindsayad/io/arfc_reactor/calc_xsecs_provided_buckling_1000_mod_1000.out'

file = open(filename, 'r')
for line in file:
    if re.search(r'Group.*Total.*Transport', line):
        break
        
file.readline()
xsecs = [float(i) for i in file.readline().split()]
print(xsecs)

    Group     Total          Transport      Absorption     Total-Scatter  Nu*Fission                                                                                         Kappa*Fission  Fission        Flux

[1.0, 0.305946, 0.26033, 0.00928483, 0.00920062, 0.00825746, 1.0126e-13, 0.00330442, 2.49795]


In [55]:
num_groups = 2

fuel_xsecs = np.ndarray(())
mod_xsecs = np.ndarray(())
for file_name in all_files:
        
    file = open(file_name, 'r')
    
    for line in file:
        if re.search(r'Group.*Total.*Transport', line):
            break
    file.readline()
    fuel_current_temp_xsecs = np.ndarray(())
    for i in range(num_groups):
        fuel_current_temp_xsecs = np.append(fuel_current_temp_xsecs, [float(i) for i in file.readline().split()], axis=0)
    fuel_xsecs = np.append(fuel_xsecs, fuel_current_temp_xsecs)
    
    for line in file:
        if re.search(r'Group.*Total.*Transport', line):
            break
    file.readline()
    mod_current_temp_xsecs = np.ndarray(())
    for i in range(num_groups):
        mod_current_temp_xsecs = np.append(mod_current_temp_xsecs, [float(i) for i in file.readline().split()])
    mod_xsecs = np.append(mod_xsecs, mod_current_temp_xsecs)

ValueError: zero-dimensional arrays cannot be concatenated

In [48]:
num_xsecs = 9
print(len(mod_temps))
print(len(mod_temps) * num_groups * num_xsecs)
print(fuel_xsecs.shape)
print(mod_xsecs.shape)

6
108
(115,)
(115,)


In [50]:
np.set_printoptions(precision=3)
print(fuel_xsecs)
print(mod_xsecs)

[  1.365e+000   0.000e+000   1.000e+000   3.179e-001   2.706e-001
   9.359e-003   9.272e-003   8.438e-003   1.035e-013   3.377e-003
   2.450e+000   2.000e+000   3.309e-001   3.178e-001   4.155e-002
   4.155e-002   6.181e-002   7.587e-013   2.475e-002   1.068e+000
   1.057e-316   1.000e+000   3.149e-001   2.681e-001   9.341e-003
   9.254e-003   8.389e-003   1.029e-013   3.357e-003   2.461e+000
   2.000e+000   3.268e-001   3.140e-001   4.024e-002   4.024e-002
   5.991e-002   7.354e-013   2.400e-002   1.101e+000   1.057e-316
   1.000e+000   3.119e-001   2.655e-001   9.317e-003   9.231e-003
   8.341e-003   1.023e-013   3.338e-003   2.473e+000   2.000e+000
   3.228e-001   3.101e-001   3.900e-002   3.900e-002   5.813e-002
   7.135e-013   2.328e-002   1.135e+000   1.057e-316   1.000e+000
   3.090e-001   2.629e-001   9.303e-003   9.218e-003   8.298e-003
   1.018e-013   3.321e-003   2.485e+000   2.000e+000   3.189e-001
   3.064e-001   3.785e-002   3.785e-002   5.648e-002   6.933e-013
   2.262e-

In [54]:
print(len(fuel_xsecs))
print(len(fuel_xsecs[0]))
print(len(fuel_xsecs[0][0]))

6
2
9


In [30]:
fuel_xsecs[0][0][0]

1.0

In [33]:
fuel_xsecs[:][0][1]

[2.0,
 0.330881,
 0.317826,
 0.0415534,
 0.0415534,
 0.0618061,
 7.58671e-13,
 0.0247541,
 1.06776]

In [34]:
fuel_xsecs[1][0][:]

[1.0,
 0.314956,
 0.268088,
 0.00932983,
 0.00924296,
 0.00836108,
 1.02528e-13,
 0.00334582,
 2.45517]

You have to think about what you're doing. I would like to have a 3-D list. The first (x) dimension is the type of cross section. The second (y) dimension is the temperature. I would like the third (z) dimension to be the group number. Think about how this would be ordered in a flattened list with a for loop structure like this:

```python
dim_3_list = []
for temp in temperatures:
    dim_2_list = []
    for group in num_groups:
        dim_2_list.append(dim_1_list)
```

In the above formulation, the first dimension is temperature, second dimension is group, and third dimension is `xsec_type`

In [11]:
all_files

['/home/lindsayad/scale/C834MNYCP02/SCALE-6.2.1/lindsayad/io/arfc_reactor/calc_xsecs_provided_buckling_800_mod_800.out',
 '/home/lindsayad/scale/C834MNYCP02/SCALE-6.2.1/lindsayad/io/arfc_reactor/calc_xsecs_provided_buckling_850_mod_800.out',
 '/home/lindsayad/scale/C834MNYCP02/SCALE-6.2.1/lindsayad/io/arfc_reactor/calc_xsecs_provided_buckling_900_mod_800.out',
 '/home/lindsayad/scale/C834MNYCP02/SCALE-6.2.1/lindsayad/io/arfc_reactor/calc_xsecs_provided_buckling_950_mod_800.out',
 '/home/lindsayad/scale/C834MNYCP02/SCALE-6.2.1/lindsayad/io/arfc_reactor/calc_xsecs_provided_buckling_1000_mod_800.out',
 '/home/lindsayad/scale/C834MNYCP02/SCALE-6.2.1/lindsayad/io/arfc_reactor/calc_xsecs_provided_buckling_1050_mod_800.out',
 '/home/lindsayad/scale/C834MNYCP02/SCALE-6.2.1/lindsayad/io/arfc_reactor/calc_xsecs_provided_buckling_800_mod_850.out',
 '/home/lindsayad/scale/C834MNYCP02/SCALE-6.2.1/lindsayad/io/arfc_reactor/calc_xsecs_provided_buckling_850_mod_850.out',
 '/home/lindsayad/scale/C834MN

In [17]:
cross_sections = [i for i in input.readline()]

In [18]:
print(cross_sections)

[' ', ' ', ' ', ' ', ' ', ' ', '1', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '3', '.', '0', '5', '9', '4', '6', 'E', '-', '0', '1', ' ', ' ', ' ', ' ', '2', '.', '6', '0', '3', '3', '0', 'E', '-', '0', '1', ' ', ' ', ' ', ' ', '9', '.', '2', '8', '4', '8', '3', 'E', '-', '0', '3', ' ', ' ', ' ', ' ', '9', '.', '2', '0', '0', '6', '2', 'E', '-', '0', '3', ' ', ' ', ' ', ' ', '8', '.', '2', '5', '7', '4', '6', 'E', '-', '0', '3', ' ', ' ', ' ', ' ', '1', '.', '0', '1', '2', '6', '0', 'E', '-', '1', '3', ' ', ' ', ' ', ' ', '3', '.', '3', '0', '4', '4', '2', 'E', '-', '0', '3', ' ', ' ', ' ', ' ', '2', '.', '4', '9', '7', '9', '5', 'E', '+', '0', '0', '\n']
